In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd

import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import pickle
from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.rpn.bbox_transform import clip_boxes
# from model.nms.nms_wrapper import nms
from model.roi_layers import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet

import pdb

try:
    xrange          # Python 2
except NameError:
    xrange = range  # Python 3

In [2]:
def save_weight(weight, time, seen):
  time = np.where(time==0, 1, time)
  weight = weight/time[:,np.newaxis]
  result_map = np.zeros((len(weight), len(weight)))
  for i in range(len(weight)):
    for j in range(len(weight)):
      v1 = weight[i]
      v2 = weight[j]
      # v1_ = np.linalg.norm(v1)
      # v2_ = np.linalg.norm(v2)
      # v12 = np.sum(v1*v2)
      # print(v12)
      # print(v1_)
      # print(v2_)
      distance = np.linalg.norm(v1-v2)
      if np.sum(v1*v2)== 0 :
        result_map[i][j] = 0
      else:
        result_map[i][j] = distance
      

  df = pd.DataFrame (result_map)

  ## save to xlsx file

  filepath = 'similarity_%d.xlsx'%(seen)

  df.to_excel(filepath, index=False)

  weight = weight*255


  cv2.imwrite('./weight_%d.png'%(seen), weight)


In [3]:
lr = cfg.TRAIN.LEARNING_RATE
momentum = cfg.TRAIN.MOMENTUM
weight_decay = cfg.TRAIN.WEIGHT_DECAY

np.random.seed(cfg.RNG_SEED)

cfg.TRAIN.USE_FLIPPED = False
dataset_name='coco'
#imdb_name = "voc_2007_trainval"
imdb_name='coco_2014_train'
#imdbval_name = "voc_2007_test"
imdbval_name = 'coco_2014_minival'
#set_cfgs = ['ANCHOR_SCALES', '[8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]']
set_cfgs = ['ANCHOR_SCALES', '[4, 8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]']
net_name='res50'
group_name=1
cfg_file = "cfgs/{}_{}.yml".format(net_name, group_name) if group_name != 0 else "cfgs/{}.yml".format(net_name)
cfg_from_file(cfg_file)
cfg_from_list(set_cfgs)
print('Using config:')
pprint.pprint(cfg)

Using config:
{'ANCHOR_RATIOS': [0.5, 1, 2],
 'ANCHOR_SCALES': [4, 8, 16, 32],
 'CROP_RESIZE_WITH_MAX_POOL': True,
 'CUDA': False,
 'DATA_DIR': '/workspace/data',
 'DEDUP_BOXES': 0.0625,
 'EPS': 1e-14,
 'EXP_DIR': 'res50',
 'FEAT_STRIDE': [16],
 'GPU_ID': 0,
 'MATLAB': 'matlab',
 'MAX_NUM_GT_BOXES': 20,
 'MOBILENET': {'DEPTH_MULTIPLIER': 1.0,
               'FIXED_LAYERS': 5,
               'REGU_DEPTH': False,
               'WEIGHT_DECAY': 4e-05},
 'PIXEL_MEANS': array([[[102.9801, 115.9465, 122.7717]]]),
 'POOLING_MODE': 'align',
 'POOLING_SIZE': 7,
 'RESNET': {'FIXED_BLOCKS': 2, 'MAX_POOL': False},
 'RNG_SEED': 3,
 'ROOT_DIR': '/workspace/One-Shot-Object-Detection',
 'TEST': {'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'MODE': 'nms',
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'gt',
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 300,
          'RPN_PRE_NMS_TOP_N': 6000,
          'RPN_TOP_

/workspace/One-Shot-Object-Detection/lib/model/utils/config.py:382: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [4]:
# Load dataset
cfg.TRAIN.USE_FLIPPED = False
imdb_vu, roidb_vu, ratio_list_vu, ratio_index_vu, query_vu = combined_roidb(imdbval_name, False, seen=2)

loading annotations into memory...
Done (t=0.68s)
creating index...
index created!
Loaded dataset `coco_2014_minival` for training
Set proposal method: gt
coco_2014_minival gt roidb loaded from /workspace/data/cache/coco_2014_minival_gt_roidb.pkl
Preparing training data...
done


In [5]:
imdb_vu.competition_mode(on=True)
dataset_vu = roibatchLoader(roidb_vu, ratio_list_vu, ratio_index_vu, query_vu, 1, imdb_vu.num_classes, 
                            training=False, seen=2)

In [6]:
# initilize the network here.
class_agnostic=True
fasterRCNN = resnet(imdb_vu.classes, 50, pretrained=False, class_agnostic=class_agnostic)
fasterRCNN.create_architecture()

In [7]:
# Load checkpoint
checksession_int=1
checkepoch_int=10
checkpoint_int=1663

input_dir = 'models' + "/" + net_name + "/" + dataset_name
if not os.path.exists(input_dir):
    raise Exception('There is no input directory for loading network from ' + input_dir)

load_name = os.path.join(input_dir,
                         'faster_rcnn_{}_{}_{}.pth'.format(checksession_int, checkepoch_int, checkpoint_int))
print("load checkpoint %s" % (load_name))
checkpoint = torch.load(load_name)
fasterRCNN.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']

load checkpoint models/res50/coco/faster_rcnn_1_10_1663.pth


In [8]:
# initilize the tensor holder here.
print('load model successfully!')
im_data = torch.FloatTensor(1)
query   = torch.FloatTensor(1)
im_info = torch.FloatTensor(1)
catgory = torch.LongTensor(1)
gt_boxes = torch.FloatTensor(1)

load model successfully!


In [9]:
# ship to cuda

cfg.CUDA = True
fasterRCNN.cuda()
im_data = im_data.cuda()
query = query.cuda()
im_info = im_info.cuda()
catgory = catgory.cuda()
gt_boxes = gt_boxes.cuda()

In [10]:
# make variable
im_data = Variable(im_data)
query = Variable(query)
im_info = Variable(im_info)
catgory = Variable(catgory)
gt_boxes = Variable(gt_boxes)

In [11]:
average_topK=1

# record time
start = time.time()

# visiualization
vis = True
if vis:
    thresh = 0.05
else:
    thresh = 0.0
max_per_image = 100

# create output Directory
output_dir_vu = get_output_dir(imdb_vu, 'faster_rcnn_unseen')

fasterRCNN.eval()
for avg in range(average_topK):
    dataset_vu.query_position = avg
    dataloader_vu = torch.utils.data.DataLoader(dataset_vu, batch_size=1,shuffle=False, num_workers=0,pin_memory=True)

    data_iter_vu = iter(dataloader_vu)

    # total quantity of testing images, each images include multiple detect class
    num_images_vu = len(imdb_vu.image_index)
    num_detect = len(ratio_index_vu[0])

    all_boxes = [[[] for _ in xrange(num_images_vu)]
                for _ in xrange(imdb_vu.num_classes)]

    
    _t = {'im_detect': time.time(), 'misc': time.time()}
    if group_name != 0:
      det_file = os.path.join(output_dir_vu, 'sess%d_g%d_seen%d_%d.pkl'%(checksession_int, group_name, 2, avg))
    else:
      det_file = os.path.join(output_dir_vu, 'sess%d_seen%d_%d.pkl'%(checksession_int, 2, avg))
    print(det_file)

    if os.path.exists(det_file):
      with open(det_file, 'rb') as fid:
        all_boxes = pickle.load(fid)
    else:
      for i,index in enumerate(ratio_index_vu[0]):
        data = next(data_iter_vu)
        im_data.data.resize_(data[0].size()).copy_(data[0])
        query.data.resize_(data[1].size()).copy_(data[1])
        im_info.data.resize_(data[2].size()).copy_(data[2])
        gt_boxes.data.resize_(data[3].size()).copy_(data[3])
        catgory.data.resize_(data[4].size()).copy_(data[4])


        # Run Testing
        det_tic = time.time()
        rois, cls_prob, bbox_pred, \
        rpn_loss_cls, rpn_loss_box, \
        RCNN_loss_cls, _, RCNN_loss_bbox, \
        rois_label, weight = fasterRCNN(im_data, query, im_info, gt_boxes, catgory)


        scores = cls_prob.data
        boxes = rois.data[:, :, 1:5]

        
        # Apply bounding-box regression 
        if cfg.TEST.BBOX_REG:
            # Apply bounding-box regression deltas
            box_deltas = bbox_pred.data
            if cfg.TRAIN.BBOX_NORMALIZE_TARGETS_PRECOMPUTED:
            # Optionally normalize targets by a precomputed mean and stdev
              if class_agnostic:
                  box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                            + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                  box_deltas = box_deltas.view(1, -1, 4)
              else:
                  box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                            + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                  box_deltas = box_deltas.view(1, -1, 4 * len(imdb.classes))

            pred_boxes = bbox_transform_inv(boxes, box_deltas, 1)
            pred_boxes = clip_boxes(pred_boxes, im_info.data, 1)
        else:
            # Simply repeat the boxes, once for each class
            pred_boxes = np.tile(boxes, (1, scores.shape[1]))


        # Resize to original ratio
        pred_boxes /= data[2][0][2].item()

        # Remove batch_size dimension
        scores = scores.squeeze()
        pred_boxes = pred_boxes.squeeze()

        # Record time
        det_toc = time.time()
        detect_time = det_toc - det_tic
        misc_tic = time.time()

        # Post processing
        inds = torch.nonzero(scores>thresh).view(-1)
        if inds.numel() > 0:
          # remove useless indices
          cls_scores = scores[inds]
          cls_boxes = pred_boxes[inds, :]
          cls_dets = torch.cat((cls_boxes, cls_scores.unsqueeze(1)), 1)

          # rearrange order
          _, order = torch.sort(cls_scores, 0, True)
          cls_dets = cls_dets[order]

          # NMS
          keep = nms(cls_boxes[order, :], cls_scores[order], cfg.TEST.NMS)
          cls_dets = cls_dets[keep.view(-1).long()]
          all_boxes[catgory][index] = cls_dets.cpu().numpy()

        # Limit to max_per_image detections *over all classes*
        if max_per_image > 0:
          try:
            image_scores = all_boxes[catgory][index][:,-1]
            if len(image_scores) > max_per_image:
                image_thresh = np.sort(image_scores)[-max_per_image]

                keep = np.where(all_boxes[catgory][index][:,-1] >= image_thresh)[0]
                all_boxes[catgory][index] = all_boxes[catgory][index][keep, :]
          except:
            pass

        misc_toc = time.time()
        nms_time = misc_toc - misc_tic

        sys.stdout.write('im_detect: {:d}/{:d} {:.3f}s {:.3f}s   \r' \
            .format(i + 1, num_detect, detect_time, nms_time))
        sys.stdout.flush()

        # save test image
        if vis and i%1==0:
          im2show = cv2.imread(dataset_vu._roidb[dataset_vu.ratio_index[i]]['image'])
          im2show = vis_detections(im2show, 'shot', cls_dets.cpu().numpy(), 0.8)

          o_query = data[1][0].permute(1, 2,0).contiguous().cpu().numpy()
          o_query *= [0.229, 0.224, 0.225]
          o_query += [0.485, 0.456, 0.406]
          o_query *= 255
          o_query = o_query[:,:,::-1]

          (h,w,c) = im2show.shape
          o_query = cv2.resize(o_query, (h, h),interpolation=cv2.INTER_LINEAR)
          im2show = np.concatenate((im2show, o_query), axis=1)

          cv2.imwrite('./test_img/%d_d.png'%(i), im2show)
      
    
      with open(det_file, 'wb') as f:
          pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)
        
    print('Evaluating detections')
    imdb_vu.evaluate_detections(all_boxes, output_dir_vu) 


    end = time.time()
    print("test time: %0.4fs" % (end - start))

/workspace/One-Shot-Object-Detection/output/res50/coco_2014_minival/faster_rcnn_unseen/sess1_g1_seen2_0.pkl
Evaluating detections
Writing results json to /workspace/One-Shot-Object-Detection/output/res50/coco_2014_minival/faster_rcnn_unseen/detections_minival2014_results.json
Loading and preparing results...     
DONE (t=1.56s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=38.03s).
Accumulating evaluation results...
DONE (t=4.50s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.232
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.055
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.186
 Average Recall     (AR) 